In [47]:

import pandas as pd
import numpy as np
#for geo-spactial points
import geopandas as gpd
import movingpandas as mpd

from shapely.geometry import Point
from pyproj import CRS
#from keplergl import KeplerGl
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import scipy as sc


from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import ConvLSTM2D
from tensorflow.keras import layers
from tensorflow.keras import activations


from datetime import datetime



In [2]:
#DAY:1
file_path='D:\Internship\AIS_Data\AIS_2022_01_01.csv'
df=pd.read_csv(file_path)


#DAY:2
file_path_1='D:\Internship\AIS_Data\AIS_2022_01_02.csv'
df_2=pd.read_csv(file_path_1)


#DAY:3
file_path_2='D:\Internship\AIS_Data\AIS_2022_01_03.csv'
df_3=pd.read_csv(file_path_2)


#DAY:4
file_path_3='D:\Internship\AIS_Data\AIS_2022_01_04.csv'
df_4=pd.read_csv(file_path_3)


#DAY:5
file_path_4='D:\Internship\AIS_Data\AIS_2022_01_05.csv'
df_5=pd.read_csv(file_path_4)


#DAY:6
file_path_5='D:\Internship\AIS_Data\AIS_2022_01_06.csv'
df_6=pd.read_csv(file_path_5)


#DAY:7
file_path_6='D:\Internship\AIS_Data\AIS_2022_01_07.csv'
df_7=pd.read_csv(file_path_6)


#DAY:8
file_path_7='D:\Internship\AIS_Data\AIS_2022_01_08.csv'
df_8=pd.read_csv(file_path_7)







In [5]:
#sort by MMSI
df.sort_values(by="MMSI")

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass
6674950,0,2022-01-01T23:37:53,22.21441,-97.86972,0.0,186.7,288.0,CGC SPENCER,NaN,NWHE,55.0,15.0,82.0,12.0,4.3,55.0,A
2328532,1,2022-01-01T08:20:34,26.05738,-80.12843,0.0,76.5,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B
6857886,1,2022-01-01T14:20:54,30.17609,-89.72594,0.0,360.0,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A
5617385,1,2022-01-01T20:59:34,26.05755,-80.12843,0.0,335.8,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,1.0,33.0,B
6247758,1,2022-01-01T22:29:33,26.05745,-80.12843,0.0,348.5,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,1.0,33.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4089976,1072211352,2022-01-01T14:31:16,33.61130,-117.92655,0.1,360.0,45.0,SHAMBHALA,IMO0000000,WDG7537,37.0,NaN,30.0,6.0,NaN,NaN,B
5923557,1072211352,2022-01-01T22:04:18,33.61132,-117.92657,0.0,360.0,46.0,SHAMBHALA,IMO0000000,WDG7537,37.0,NaN,30.0,6.0,NaN,NaN,B
2397026,1072211352,2022-01-01T08:34:14,33.61134,-117.92655,0.2,360.0,45.0,SHAMBHALA,IMO0000000,WDG7537,37.0,NaN,30.0,6.0,NaN,NaN,B
1036294,1072211352,2022-01-01T03:40:14,33.61132,-117.92655,0.0,360.0,46.0,SHAMBHALA,IMO0000000,WDG7537,37.0,NaN,30.0,6.0,NaN,NaN,B


In [6]:
#Navigation Status
files_path="D:\\Internship\\AIS_Data\\navstatus.csv"
df_nav=pd.read_csv(files_path)
df_nav



,Status,vessel_status
0,0,Under way using its engine
1,1,Anchored
2,2,Not under command
3,3,Has restricted maneuverability
4,4,Ship draught is limiting its movement
5,5,Moored (tied to another object to limit free m...
6,6,Aground
7,7,Engaged in fishing
8,8,Under way sailing
9,9,Number reserved for modifying reported status ...


In [7]:
#convert the nav_status to float and then mergw with the AIS data

df_nav = df_nav.replace(r'^\s*$', np.nan, regex=True)
df_nav = df_nav.astype({'Status':'float'})
df_nav


,Status,vessel_status
0,0.0,Under way using its engine
1,1.0,Anchored
2,2.0,Not under command
3,3.0,Has restricted maneuverability
4,4.0,Ship draught is limiting its movement
5,5.0,Moored (tied to another object to limit free m...
6,6.0,Aground
7,7.0,Engaged in fishing
8,8.0,Under way sailing
9,9.0,Number reserved for modifying reported status ...


In [8]:
df=pd.merge(df,df_nav,how='left',on='Status')

df

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,vessel_status
0,368084090,2022-01-01T00:00:00,29.93174,-89.99243,6.0,296.2,299.0,LARRY B WHIPPLE,NaN,WDK7401,57.0,12.0,23.0,10.0,3.0,57.0,A,Power-driven vessel pushing ahead/towing alon...
1,368140160,2022-01-01T00:00:00,30.33475,-87.14429,0.0,312.0,87.0,TWISTED ANGEL,IMO0000000,WDL5339,36.0,NaN,12.0,7.0,NaN,NaN,B,NaN
2,366941830,2022-01-01T00:00:02,29.30919,-94.79702,0.0,180.2,511.0,SAN PATRICIO,NaN,WCX6675,31.0,5.0,18.0,7.0,NaN,57.0,A,Moored (tied to another object to limit free m...
3,316005971,2022-01-01T00:00:06,46.50268,-84.35674,2.4,258.6,257.0,BEVERLY M I,IMO9084047,CFP2004,31.0,0.0,34.0,10.0,5.3,99.0,A,Under way using its engine
4,316004054,2022-01-01T00:00:07,46.50326,-84.37506,0.3,61.9,511.0,ADANAC III,IMO8745333,VCLT,31.0,0.0,24.0,5.0,3.0,50.0,A,Under way using its engine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7239753,372188000,2022-01-01T23:20:44,13.46062,144.66641,0.0,45.2,322.0,DANCEWOOD SW,IMO9639672,3FGQ2,70.0,5.0,170.0,27.0,9.7,70.0,A,Moored (tied to another object to limit free m...
7239754,372188000,2022-01-01T23:23:44,13.46062,144.66645,0.1,57.6,322.0,DANCEWOOD SW,IMO9639672,3FGQ2,70.0,5.0,170.0,27.0,9.7,70.0,A,Moored (tied to another object to limit free m...
7239755,372188000,2022-01-01T23:29:44,13.46063,144.66643,0.1,47.7,322.0,DANCEWOOD SW,IMO9639672,3FGQ2,70.0,5.0,170.0,27.0,9.7,70.0,A,Moored (tied to another object to limit free m...
7239756,372188000,2022-01-01T23:50:44,13.46065,144.66642,0.0,232.8,322.0,DANCEWOOD SW,IMO9639672,3FGQ2,70.0,5.0,170.0,27.0,9.7,70.0,A,Moored (tied to another object to limit free m...


In [9]:
#the number of times the vessels are Anchored
test1 = df['vessel_status'].str.contains('Anchored').value_counts()
test1

False    5125050
True      245871
Name: vessel_status, dtype: int64

In [ ]:
#LOADING PORTS Dataset

In [10]:
file_path2='D:\Internship\Dataset\data\code-list_csv.csv'
df2=pd.read_csv(file_path2)
df2

,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
0,NaN,AD,ALV,Andorra la Vella,Andorra la Vella,NaN,AI,--34-6--,601.0,NaN,4230N 00131E,NaN
1,NaN,AD,CAN,Canillo,Canillo,NaN,RL,--3-----,307.0,NaN,4234N 00135E,NaN
2,NaN,AD,ENC,Encamp,Encamp,NaN,RL,--3-----,307.0,NaN,4232N 00134E,NaN
3,NaN,AD,ESC,Escaldes-Engordany,Escaldes-Engordany,NaN,RL,--3-----,307.0,NaN,4231N 00133E,NaN
4,NaN,AD,EAC,Escàs,Escas,04,RL,--3-----,1407.0,NaN,4233N 00131E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
111548,NaN,ZW,STH,Southerton,Southerton,NaN,RL,--3-----,201.0,NaN,1751S 03101E,NaN
111549,NaN,ZW,THJ,Thompson Junction,Thompson Junction,NaN,RL,-23-----,701.0,NaN,1800S 02626E,NaN
111550,NaN,ZW,VFA,Victoria Falls,Victoria Falls,NaN,AI,---4----,9501.0,NaN,NaN,NaN
111551,NaN,ZW,ZMZ,Zimbabwe,Zimbabwe,MV,RL,1-3-----,1401.0,NaN,2016S 03055E,NaN


In [11]:
df2.dtypes
US_port=df2.loc[df2['Country']=='US']

US_port


,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
89057,NaN,US,AT2,"Allentown, Allegany","Allentown, Allegany",NY,RL,--3-----,607.0,NaN,4205N 07804W,NaN
89058,NaN,US,JIN,"Alpine, San Diego","Alpine, San Diego",CA,RL,--3-----,901.0,NaN,3250N 11646W,NaN
89059,NaN,US,ZJI,"Alpine, Los Angeles","Alpine, Los Angeles",CA,RL,-23--6--,901.0,NaN,3432N 11806W,NaN
89060,NaN,US,AGW,"Arlington, Houston","Arlington, Houston",TN,RQ,--3--6--,607.0,NaN,3619N 08743W,NaN
89061,NaN,US,AT3,"Arlington, Knox","Arlington, Knox",TN,RQ,--3--6--,607.0,NaN,3600N 08357W,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
109890,NaN,US,Z9T,Zitlaltepec,Zitlaltepec,NM,RL,--3-----,1501.0,NaN,1912N 09754W,NaN
109891,NaN,US,ZPS,Zolfo Springs,Zolfo Springs,FL,RL,-----6--,901.0,NaN,2729N 08147W,NaN
109892,NaN,US,ZUL,Zullinger,Zullinger,PA,RL,--3-----,1307.0,NaN,3946N 07737W,NaN
109893,NaN,US,ZUM,Zumbrota,Zumbrota,MN,RL,1-3--6--,307.0,NaN,4417N 09240W,NaN


In [12]:
#Locations that have PORTS :
id_ports=US_port[US_port['Function'].str.contains("1")]
id_ports

,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks
89088,NaN,US,CS6,"Coxsackie, Greene","Coxsackie, Greene",NY,RL,123-----,607.0,NaN,4221N 07348W,NaN
89089,NaN,US,USB,"Curtis Bay, Baltimore","Curtis Bay, Baltimore",MD,RL,123-----,601.0,NaN,3913N 07635W,NaN
89094,NaN,US,DOG,"Douglas, Allegan","Douglas, Allegan",MI,RL,1-3--6--,501.0,NaN,4238N 08612W,NaN
89107,NaN,US,FMW,"Fairmont, Will","Fairmont, Will",IL,RL,123--6--,501.0,NaN,4133N 08803W,NaN
89110,NaN,US,ZFI,"Fisher Island, Miami","Fisher Island, Miami",FL,RL,1-3-----,607.0,NaN,2546N 08008W,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
109829,NaN,US,ZYH,York Harbor,York Harbor,ME,RL,1-3--6--,307.0,NaN,4308N 07038W,NaN
109846,NaN,US,YTW,Youngstown,Youngstown,NY,RL,1-3-----,407.0,NaN,4314N 07903W,NaN
109867,NaN,US,8PR,Zamora,Zamora,CA,RL,1-3-----,1401.0,NaN,3847N 12152W,NaN
109888,NaN,US,ZR9,Zirconia,Zirconia,NC,RL,1----6--,1201.0,NaN,3514N 08224W,NaN


In [13]:
#b=id_ports['Coordinates'].dtypes
#b
id_ports[['LAT', 'LON']] = id_ports['Coordinates'].str.split(' ', expand=True)
id_ports

C:\Users\saisu\AppData\Local\Temp\ipykernel_25156\4082824808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_ports[['LAT', 'LON']] = id_ports['Coordinates'].str.split(' ', expand=True)
C:\Users\saisu\AppData\Local\Temp\ipykernel_25156\4082824808.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_ports[['LAT', 'LON']] = id_ports['Coordinates'].str.split(' ', expand=True)


,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,Coordinates,Remarks,LAT,LON
89088,NaN,US,CS6,"Coxsackie, Greene","Coxsackie, Greene",NY,RL,123-----,607.0,NaN,4221N 07348W,NaN,4221N,07348W
89089,NaN,US,USB,"Curtis Bay, Baltimore","Curtis Bay, Baltimore",MD,RL,123-----,601.0,NaN,3913N 07635W,NaN,3913N,07635W
89094,NaN,US,DOG,"Douglas, Allegan","Douglas, Allegan",MI,RL,1-3--6--,501.0,NaN,4238N 08612W,NaN,4238N,08612W
89107,NaN,US,FMW,"Fairmont, Will","Fairmont, Will",IL,RL,123--6--,501.0,NaN,4133N 08803W,NaN,4133N,08803W
89110,NaN,US,ZFI,"Fisher Island, Miami","Fisher Island, Miami",FL,RL,1-3-----,607.0,NaN,2546N 08008W,NaN,2546N,08008W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109829,NaN,US,ZYH,York Harbor,York Harbor,ME,RL,1-3--6--,307.0,NaN,4308N 07038W,NaN,4308N,07038W
109846,NaN,US,YTW,Youngstown,Youngstown,NY,RL,1-3-----,407.0,NaN,4314N 07903W,NaN,4314N,07903W
109867,NaN,US,8PR,Zamora,Zamora,CA,RL,1-3-----,1401.0,NaN,3847N 12152W,NaN,3847N,12152W
109888,NaN,US,ZR9,Zirconia,Zirconia,NC,RL,1----6--,1201.0,NaN,3514N 08224W,NaN,3514N,08224W


In [14]:
id_ports['LAT_DD']=id_ports['LAT'].str[:2]
id_ports['LAT_MM']=id_ports['LAT'].str[2:4]
id_ports['LAT_NS']=id_ports['LAT'].str[-1:]

id_ports['LON_DD']=id_ports['LON'].str[:3]
id_ports['LON_MM']=id_ports['LON'].str[3:5]
id_ports['LON_EW']=id_ports['LON'].str[-1:] 


#Convert to float
id_ports = id_ports.astype({'LAT_DD':'float','LAT_MM':'float','LON_DD':'float','LON_MM':'float'})


# aa=id_ports.max()
# aa
#Convert to degree
id_ports['LAT_DEG']=id_ports['LAT_DD']+(id_ports['LAT_MM']/100 )
id_ports['LON_DEG']=id_ports['LON_DD']+(id_ports['LON_MM']/100 )
id_ports




#convert with proper sign (N/S AND W/E)
#id_ports['LAT_DEG']











C:\Users\saisu\AppData\Local\Temp\ipykernel_25156\3355816322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_ports['LAT_DD']=id_ports['LAT'].str[:2]
C:\Users\saisu\AppData\Local\Temp\ipykernel_25156\3355816322.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_ports['LAT_MM']=id_ports['LAT'].str[2:4]
C:\Users\saisu\AppData\Local\Temp\ipykernel_25156\3355816322.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,...,LAT,LON,LAT_DD,LAT_MM,LAT_NS,LON_DD,LON_MM,LON_EW,LAT_DEG,LON_DEG
89088,NaN,US,CS6,"Coxsackie, Greene","Coxsackie, Greene",NY,RL,123-----,607.0,NaN,...,4221N,07348W,42.0,21.0,N,73.0,48.0,W,42.21,73.48
89089,NaN,US,USB,"Curtis Bay, Baltimore","Curtis Bay, Baltimore",MD,RL,123-----,601.0,NaN,...,3913N,07635W,39.0,13.0,N,76.0,35.0,W,39.13,76.35
89094,NaN,US,DOG,"Douglas, Allegan","Douglas, Allegan",MI,RL,1-3--6--,501.0,NaN,...,4238N,08612W,42.0,38.0,N,86.0,12.0,W,42.38,86.12
89107,NaN,US,FMW,"Fairmont, Will","Fairmont, Will",IL,RL,123--6--,501.0,NaN,...,4133N,08803W,41.0,33.0,N,88.0,3.0,W,41.33,88.03
89110,NaN,US,ZFI,"Fisher Island, Miami","Fisher Island, Miami",FL,RL,1-3-----,607.0,NaN,...,2546N,08008W,25.0,46.0,N,80.0,8.0,W,25.46,80.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109829,NaN,US,ZYH,York Harbor,York Harbor,ME,RL,1-3--6--,307.0,NaN,...,4308N,07038W,43.0,8.0,N,70.0,38.0,W,43.08,70.38
109846,NaN,US,YTW,Youngstown,Youngstown,NY,RL,1-3-----,407.0,NaN,...,4314N,07903W,43.0,14.0,N,79.0,3.0,W,43.14,79.03
109867,NaN,US,8PR,Zamora,Zamora,CA,RL,1-3-----,1401.0,NaN,...,3847N,12152W,38.0,47.0,N,121.0,52.0,W,38.47,121.52
109888,NaN,US,ZR9,Zirconia,Zirconia,NC,RL,1----6--,1201.0,NaN,...,3514N,08224W,35.0,14.0,N,82.0,24.0,W,35.14,82.24


In [15]:
# if "W" in id_ports['LON_EW'].values:
#      id_ports['LON_DEG']= id_ports['LON_DEG']*-1
# else:
#     id_ports['LON_DEG']= id_ports['LON_DEG']

# id_ports['LON_DEG'] = id_ports['LON_EW'].apply( lambda x: id_ports['LON_DEG'] if x=="E" else id_ports['LON_DEG']*-1 )


def condition(x):
    if x=="E":
        return 1
    else:
        return -1

id_ports['LON_DEG'] = id_ports['LON_EW'].apply(condition)*id_ports['LON_DEG']
 
id_ports


,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,...,LAT,LON,LAT_DD,LAT_MM,LAT_NS,LON_DD,LON_MM,LON_EW,LAT_DEG,LON_DEG
89088,NaN,US,CS6,"Coxsackie, Greene","Coxsackie, Greene",NY,RL,123-----,607.0,NaN,...,4221N,07348W,42.0,21.0,N,73.0,48.0,W,42.21,-73.48
89089,NaN,US,USB,"Curtis Bay, Baltimore","Curtis Bay, Baltimore",MD,RL,123-----,601.0,NaN,...,3913N,07635W,39.0,13.0,N,76.0,35.0,W,39.13,-76.35
89094,NaN,US,DOG,"Douglas, Allegan","Douglas, Allegan",MI,RL,1-3--6--,501.0,NaN,...,4238N,08612W,42.0,38.0,N,86.0,12.0,W,42.38,-86.12
89107,NaN,US,FMW,"Fairmont, Will","Fairmont, Will",IL,RL,123--6--,501.0,NaN,...,4133N,08803W,41.0,33.0,N,88.0,3.0,W,41.33,-88.03
89110,NaN,US,ZFI,"Fisher Island, Miami","Fisher Island, Miami",FL,RL,1-3-----,607.0,NaN,...,2546N,08008W,25.0,46.0,N,80.0,8.0,W,25.46,-80.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109829,NaN,US,ZYH,York Harbor,York Harbor,ME,RL,1-3--6--,307.0,NaN,...,4308N,07038W,43.0,8.0,N,70.0,38.0,W,43.08,-70.38
109846,NaN,US,YTW,Youngstown,Youngstown,NY,RL,1-3-----,407.0,NaN,...,4314N,07903W,43.0,14.0,N,79.0,3.0,W,43.14,-79.03
109867,NaN,US,8PR,Zamora,Zamora,CA,RL,1-3-----,1401.0,NaN,...,3847N,12152W,38.0,47.0,N,121.0,52.0,W,38.47,-121.52
109888,NaN,US,ZR9,Zirconia,Zirconia,NC,RL,1----6--,1201.0,NaN,...,3514N,08224W,35.0,14.0,N,82.0,24.0,W,35.14,-82.24


In [16]:
a=id_ports[id_ports['LON_EW']=="E"]
a

,Change,Country,Location,Name,NameWoDiacritics,Subdivision,Status,Function,Date,IATA,...,LAT,LON,LAT_DD,LAT_MM,LAT_NS,LON_DD,LON_MM,LON_EW,LAT_DEG,LON_DEG
96182,NaN,US,GBV,Garberville,Garberville,CA,RL,1-3--6--,601.0,NaN,...,4006N,12348E,40.0,6.0,N,123.0,48.0,E,40.06,123.48
96329,NaN,US,TUV,Gertrude Cove,Gertrude Cove,AK,RQ,1-------,901.0,NaN,...,5156N,17727E,51.0,56.0,N,177.0,27.0,E,51.56,177.27
98970,NaN,US,KAB,Kiska Harbour,Kiska Harbour,AK,RL,1-------,1201.0,NaN,...,5158N,17734E,51.0,58.0,N,177.0,34.0,E,51.58,177.34
98971,NaN,US,KIS,Kiska Island,Kiska Island,AK,RL,1-------,212.0,NaN,...,5157N,17733E,51.0,57.0,N,177.0,33.0,E,51.57,177.33


In [17]:
df['BaseDateTime'] = pd.to_datetime(df['BaseDateTime'])
aa=df.dtypes
aa

MMSI                         int64
BaseDateTime        datetime64[ns]
LAT                        float64
LON                        float64
SOG                        float64
COG                        float64
Heading                    float64
VesselName                  object
IMO                         object
CallSign                    object
VesselType                 float64
Status                     float64
Length                     float64
Width                      float64
Draft                      float64
Cargo                      float64
TransceiverClass            object
vessel_status               object
dtype: object

In [18]:
df.describe()[['SOG']]

,SOG
count,7.239758e+06
mean,2.073895e+00
std,5.928803e+00
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,5.000000e-01
max,1.023000e+02


In [19]:
df.isnull().sum(axis = 0)

MMSI                      0
BaseDateTime              0
LAT                       0
LON                       0
SOG                       0
COG                       0
Heading                   0
VesselName            17546
IMO                 3199337
CallSign             893918
VesselType             1920
Status              1868837
Length               449765
Width               1178976
Draft               4469706
Cargo               1866011
TransceiverClass          0
vessel_status       1868837
dtype: int64

In [20]:
id_ports.dtypes

Change               object
Country              object
Location             object
Name                 object
NameWoDiacritics     object
Subdivision          object
Status               object
Function             object
Date                float64
IATA                 object
Coordinates          object
Remarks              object
LAT                  object
LON                  object
LAT_DD              float64
LAT_MM              float64
LAT_NS               object
LON_DD              float64
LON_MM              float64
LON_EW               object
LAT_DEG             float64
LON_DEG             float64
dtype: object

In [21]:
df_nav.dtypes

Status           float64
vessel_status     object
dtype: object

In [22]:
id_ports["LON_EW"].value_counts()

W    1536
E       4
Name: LON_EW, dtype: int64

In [23]:
df["Heading"].isna().sum()

0

In [24]:
frames=[df,df_2,df_3,df_4,df_5,df_6,df_7]
df_fi=pd.concat(frames)

In [25]:
a=df_fi.shape
a

(49084624, 18)

In [26]:
df['MMSI'].nunique()

14778

In [27]:
#To find max and min for with MMSI So that we can find which vessel is moving
#DAY1
df_max_d1 = df.groupby(['MMSI'])['LAT'].agg('max')


df_min_d1= df.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d1 = pd.merge(df_max_d1,df_min_d1,how='left',on='MMSI')
df_max_min_d1





,LAT_x,LAT_y
MMSI,,
0,22.21441,22.21441
1,30.17663,26.05735
11,27.29239,27.29206
111,27.35405,27.35354
737255,18.45958,18.45942
...,...,...
985346207,29.68796,29.68789
985380357,18.33428,18.31683
998975541,30.30266,30.28461


In [28]:
#DAY2
df_max_d2 = df_2.groupby(['MMSI'])['LAT'].agg('max')


df_min_d2= df_2.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d2 = pd.merge(df_max_d2,df_min_d2,how='left',on='MMSI')
df_max_min_d2

,LAT_x,LAT_y
MMSI,,
0,22.21442,22.21439
11,27.29232,27.29221
111,27.35386,27.35358
3660489,27.37037,27.37008
3660512,26.12889,26.12883
...,...,...
985380029,24.59698,24.56192
985380950,30.51273,30.47263
998975541,30.30018,30.28457


In [29]:
#DAY3
df_max_d3 = df_3.groupby(['MMSI'])['LAT'].agg('max')


df_min_d3= df_3.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d3 = pd.merge(df_max_d3,df_min_d3,how='left',on='MMSI')
df_max_min_d3

,LAT_x,LAT_y
MMSI,,
0,46.28138,46.28138
507027,39.82339,39.82317
3660489,27.37023,27.37013
3669881,43.26186,43.26175
3669882,42.87713,42.87708
...,...,...
985380950,30.51285,30.51257
985380979,26.09772,26.07732
998975541,30.28481,30.28465


In [30]:
#DAY4
df_max_d4 = df_4.groupby(['MMSI'])['LAT'].agg('max')


df_min_d4= df_4.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d4 = pd.merge(df_max_d4,df_min_d4,how='left',on='MMSI')
df_max_min_d4

,LAT_x,LAT_y
MMSI,,
0,36.88184,36.88085
1,30.17667,30.02173
11,27.29237,27.29222
111,27.35394,27.35357
507027,39.82352,39.82314
...,...,...
985380065,24.55842,24.55724
985380421,26.05972,26.05889
998975541,30.28489,30.18669


In [31]:
#DAY5
df_max_d5 = df_5.groupby(['MMSI'])['LAT'].agg('max')


df_min_d5= df_5.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d5 = pd.merge(df_max_d5,df_min_d5,how='left',on='MMSI')
df_max_min_d5

,LAT_x,LAT_y
MMSI,,
0,36.88230,36.88075
1,39.75669,30.02308
11,27.29224,27.29224
507027,39.82353,39.82309
3660489,27.37032,27.37005
...,...,...
985346207,29.68795,29.68790
985380200,32.65789,32.65789
998975541,30.28478,30.28453


In [32]:
#DAY6
df_max_d6 = df_6.groupby(['MMSI'])['LAT'].agg('max')


df_min_d6= df_6.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d6 = pd.merge(df_max_d6,df_min_d6,how='left',on='MMSI')
df_max_min_d6

,LAT_x,LAT_y
MMSI,,
0,36.88220,36.88057
1,30.17666,30.01341
368926,32.67932,32.67932
507027,39.82344,39.82305
3660489,27.37033,27.37012
...,...,...
986486000,26.77705,26.77701
998975541,30.28482,30.28468
999999999,36.93592,36.93537


In [33]:
#DAY7
df_max_d7 = df_7.groupby(['MMSI'])['LAT'].agg('max')


df_min_d7= df_7.groupby(['MMSI'])['LAT'].agg('min')


df_max_min_d7 = pd.merge(df_max_d7,df_min_d7,how='left',on='MMSI')
df_max_min_d7

,LAT_x,LAT_y
MMSI,,
0,36.88225,36.88042
1,30.17669,30.01393
111,27.35379,27.35365
507027,39.82343,39.82317
3660489,27.37023,27.37000
...,...,...
985380421,26.12309,26.05895
998975541,30.71421,29.10088
999999999,36.93594,36.93361


In [34]:
df_fi_MaxMin=pd.concat([df_max_min_d1,df_max_min_d2,df_max_min_d3,df_max_min_d4,df_max_min_d5,df_max_min_d6,df_max_min_d7])
df_fi_MaxMin

,LAT_x,LAT_y
MMSI,,
0,22.21441,22.21441
1,30.17663,26.05735
11,27.29239,27.29206
111,27.35405,27.35354
737255,18.45958,18.45942
...,...,...
985380421,26.12309,26.05895
998975541,30.71421,29.10088
999999999,36.93594,36.93361


In [35]:
df_max_dd = df_fi_MaxMin.groupby(['MMSI'])['LAT_x'].agg('max')


df_min_dd= df_fi_MaxMin.groupby(['MMSI'])['LAT_y'].agg('min')


df_max_min = pd.merge(df_max_dd,df_min_dd,how='left',on='MMSI')
df_max_min

,LAT_x,LAT_y
MMSI,,
0,46.28138,22.21439
1,39.75669,26.05735
11,27.29239,27.29206
111,27.35405,27.35354
368926,32.67932,32.67932
...,...,...
986486000,26.77705,26.77701
998975541,30.71421,29.10088
999999999,36.93602,26.09388


In [36]:
df_d1=df.loc[df['MMSI']==1]

df_d2=df_2.loc[df_2['MMSI']==1]

df_d3=df_3.loc[df_3['MMSI']==1]

df_d4=df_4.loc[df_4['MMSI']==1]

df_d5=df_5.loc[df_5['MMSI']==1]

df_d6=df_6.loc[df_6['MMSI']==1]

df_d7=df_7.loc[df_7['MMSI']==1]

df_fi_d1=pd.concat([df_d1,df_d2,df_d3,df_d4,df_d5,df_d6,df_d7])

df_fi_d1

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,vessel_status
23280,1,2022-01-01 00:02:33,26.05740,-80.12843,0.0,283.2,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN
37280,1,2022-01-01 00:05:33,26.05740,-80.12842,0.0,293.1,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN
48044,1,2022-01-01 00:08:33,26.05742,-80.12839,0.0,289.4,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN
173933,1,2022-01-01 00:35:34,26.05749,-80.12840,0.0,256.0,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN
252317,1,2022-01-01 00:29:34,26.05744,-80.12838,0.0,254.1,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6798950,1,2022-01-07T19:12:57,30.12022,-89.68505,36.1,59.1,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN
6810077,1,2022-01-07T19:15:05,30.13151,-89.66354,38.8,57.9,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN
6848388,1,2022-01-07T19:19:17,30.15304,-89.62996,28.3,321.7,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN
6857301,1,2022-01-07T19:22:27,30.16284,-89.65879,40.1,306.9,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN


In [37]:
df_fi_d1['geometry'] = [Point(long, lat) for long, lat in zip(df_fi_d1['LON'].to_list(),df_fi_d1['LAT'].to_list())]
df_fi_d1


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,vessel_status,geometry
23280,1,2022-01-01 00:02:33,26.05740,-80.12843,0.0,283.2,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN,POINT (-80.12842999999999 26.0574)
37280,1,2022-01-01 00:05:33,26.05740,-80.12842,0.0,293.1,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN,POINT (-80.12842000000001 26.0574)
48044,1,2022-01-01 00:08:33,26.05742,-80.12839,0.0,289.4,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN,POINT (-80.12839 26.05742)
173933,1,2022-01-01 00:35:34,26.05749,-80.12840,0.0,256.0,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN,POINT (-80.1284 26.05749)
252317,1,2022-01-01 00:29:34,26.05744,-80.12838,0.0,254.1,511.0,MISS MARGARET,IMO0000000,NaN,37.0,NaN,21.0,6.0,0.0,0.0,B,NaN,POINT (-80.12838000000001 26.05744)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6798950,1,2022-01-07T19:12:57,30.12022,-89.68505,36.1,59.1,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN,POINT (-89.68505 30.12022)
6810077,1,2022-01-07T19:15:05,30.13151,-89.66354,38.8,57.9,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN,POINT (-89.66354 30.13151)
6848388,1,2022-01-07T19:19:17,30.15304,-89.62996,28.3,321.7,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN,POINT (-89.62996 30.15304)
6857301,1,2022-01-07T19:22:27,30.16284,-89.65879,40.1,306.9,511.0,MISS MARGARET,IMO0000000,NaN,37.0,0.0,21.0,6.0,1.0,33.0,A,NaN,POINT (-89.65879 30.16284)


In [ ]:
#GEOPANDAS AND MOVINGPANDAS

In [38]:
geodata = gpd.GeoDataFrame(df_fi_d1, crs = CRS.from_epsg('4326'))

In [39]:
geodata['t'] = pd.to_datetime(geodata['BaseDateTime']).dt.tz_localize(None)
# Set timestamp as Index
geodata = geodata.set_index('t')
geodata.head()
# geodata_subset = geodata[['t', 'geometry']]
# geodata_subset.head()
traj_collection = mpd.TrajectoryCollection(geodata, 'MMSI')

traj_collection.hvplot(geo=True, tiles='OSM', line_width=5, frame_width=500, frame_height=300)


C:\Users\saisu\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:105: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  super(GeoDataFrame, self).__init__(*args, **kwargs)


:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

In [ ]:
#Hours = 12
#SearchRadio = 1000
# stop detection
#stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), max_diameter=SearchRadio)

In [40]:
df_fi_d1['LON'].value_counts()

-89.72576    36
-89.72577    34
-89.72578    33
-89.76510    27
-89.76541    24
             ..
-89.63399     1
-89.73406     1
-89.64920     1
-89.75176     1
-89.65879     1
Name: LON, Length: 819, dtype: int64

In [41]:
df_d11=df.loc[df['MMSI']==1072339762]

df_d22=df_2.loc[df_2['MMSI']==1072339762]

df_d33=df_3.loc[df_3['MMSI']==1072339762]

df_d44=df_4.loc[df_4['MMSI']==1072339762]

df_d55=df_5.loc[df_5['MMSI']==1072339762]

df_d66=df_6.loc[df_6['MMSI']==1072339762]

df_d77=df_7.loc[df_7['MMSI']==1072339762]

df_fi_d1072339762=pd.concat([df_d11,df_d22,df_d33,df_d44,df_d55,df_d66,df_d77])

df_fi_d1072339762

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,vessel_status
4735813,1072339762,2022-01-06T16:49:46,24.55903,-81.73685,0.0,360.0,275.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
4845591,1072339762,2022-01-06T17:08:42,24.55903,-81.73685,0.0,360.0,275.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
4872130,1072339762,2022-01-06T17:17:44,24.55902,-81.73684,0.0,360.0,275.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
4920392,1072339762,2022-01-06T17:23:43,24.55903,-81.73683,0.0,360.0,274.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
5071444,1072339762,2022-01-06T17:56:45,24.55903,-81.73685,0.0,360.0,273.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6408365,1072339762,2022-01-07T18:13:01,24.56023,-81.73642,0.0,360.0,94.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
6534606,1072339762,2022-01-07T18:34:01,24.56046,-81.73533,5.0,15.3,8.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
6581240,1072339762,2022-01-07T18:40:07,24.56528,-81.73631,1.9,174.5,352.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN
6585202,1072339762,2022-01-07T18:42:08,24.56515,-81.73672,0.2,360.0,90.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN


In [42]:
df_fi_d1072339762['geometry'] = [Point(long, lat) for long, lat in zip(df_fi_d1072339762['LON'].to_list(),df_fi_d1072339762['LAT'].to_list())]
df_fi_d1072339762

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,IMO,CallSign,VesselType,Status,Length,Width,Draft,Cargo,TransceiverClass,vessel_status,geometry
4735813,1072339762,2022-01-06T16:49:46,24.55903,-81.73685,0.0,360.0,275.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73685 24.55903)
4845591,1072339762,2022-01-06T17:08:42,24.55903,-81.73685,0.0,360.0,275.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73685 24.55903)
4872130,1072339762,2022-01-06T17:17:44,24.55902,-81.73684,0.0,360.0,275.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73684 24.55902)
4920392,1072339762,2022-01-06T17:23:43,24.55903,-81.73683,0.0,360.0,274.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73683 24.55903)
5071444,1072339762,2022-01-06T17:56:45,24.55903,-81.73685,0.0,360.0,273.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73685 24.55903)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6408365,1072339762,2022-01-07T18:13:01,24.56023,-81.73642,0.0,360.0,94.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73642 24.56023)
6534606,1072339762,2022-01-07T18:34:01,24.56046,-81.73533,5.0,15.3,8.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73533 24.56046)
6581240,1072339762,2022-01-07T18:40:07,24.56528,-81.73631,1.9,174.5,352.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73631 24.56528)
6585202,1072339762,2022-01-07T18:42:08,24.56515,-81.73672,0.2,360.0,90.0,WQ52,IMO0000000,NaN,37.0,NaN,18.0,3.0,NaN,NaN,B,NaN,POINT (-81.73672000000001 24.56515)


In [43]:
# df_fi_d1072339762['LAT_pred']=df_fi_d1['LAT'].interpolate(method='spline',order =2)
# df_fi_d1072339762['LON_pred']=df_fi_d1['LON'].interpolate(method='spline',order =2)
# df_fi_d1072339762['geometry_pred'] = [Point(long, lat) for long, lat in zip(df_fi_d1['LON_pred'].to_list(),df_fi_d1['LAT_pred'].to_list())]

geodata1 = gpd.GeoDataFrame(df_fi_d1072339762, crs = CRS.from_epsg('4326'))
geodata1['t'] = pd.to_datetime(geodata1['BaseDateTime']).dt.tz_localize(None)
# Set timestamp as Index
geodata1 = geodata1.set_index('t')
geodata1.head()
# geodata_subset = geodata[['t', 'geometry']]
# geodata_subset.head()


traj_collection1 = mpd.TrajectoryCollection(geodata1, 'MMSI')

traj_collection1.hvplot(geo=True, tiles='OSM', line_width=5, frame_width=500, frame_height=300)


:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]

In [44]:
from numpy import array
 
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
            # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)
 
# define input sequence
raw_seq = df_fi_d1072339762['LAT']
# choose a number of time steps
raw_seq.to_numpy()
raw_seq2=pd.Series(raw_seq).array

print(type(raw_seq2))

n_steps = 4
# split into samples
X, y = split_sequence(raw_seq2, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
    
len(raw_seq2)

<class 'pandas.core.arrays.numpy_.PandasArray'>
[24.55903 24.55903 24.55902 24.55903] 24.55903
[24.55903 24.55902 24.55903 24.55903] 24.55902
[24.55902 24.55903 24.55903 24.55902] 24.55901
[24.55903 24.55903 24.55902 24.55901] 24.559
[24.55903 24.55902 24.55901 24.559  ] 24.55904
[24.55902 24.55901 24.559   24.55904] 24.55899
[24.55901 24.559   24.55904 24.55899] 24.55899
[24.559   24.55904 24.55899 24.55899] 24.55902
[24.55904 24.55899 24.55899 24.55902] 24.55903
[24.55899 24.55899 24.55902 24.55903] 24.55902
[24.55899 24.55902 24.55903 24.55902] 24.55904
[24.55902 24.55903 24.55902 24.55904] 24.559
[24.55903 24.55902 24.55904 24.559  ] 24.55905
[24.55902 24.55904 24.559   24.55905] 24.55902
[24.55904 24.559   24.55905 24.55902] 24.55904
[24.559   24.55905 24.55902 24.55904] 24.55904
[24.55905 24.55902 24.55904 24.55904] 24.55902
[24.55902 24.55904 24.55904 24.55902] 24.55905
[24.55904 24.55904 24.55902 24.55905] 24.55901
[24.55904 24.55902 24.55905 24.55901] 24.55902
[24.55902 24.559

87

In [48]:


n_features = 1
n_seq = 2
n_steps = 2
X = X.reshape((X.shape[0], n_seq, 1, n_steps, n_features))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(layers.Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=500, verbose=0)

x_input = X[(len(raw_seq2)-n_steps-3)]
x_input = x_input.reshape((1, n_seq, 1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[24.562563]]


In [ ]:
df_fi_d1072339762_111=df_fi_d1072339762.loc[df_fi_d1072339762['Status']=='1']
df_fi_d1072339762_111

In [ ]:
#df_fi_d1072339762['TimeStamp'] = (df_fi_d1072339762['BaseDateTime'] - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')

In [ ]:
#LONGTITUDE:

In [49]:
# split a univariate sequence into samples
def split_sequence_LON(sequence, n_steps):
    X_LON, y_LON = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
            # gather input and output parts of the pattern
        seq_x_LON, seq_y_LON = sequence[i:end_ix], sequence[end_ix]
        X_LON.append(seq_x_LON)
        y_LON.append(seq_y_LON)
    return array(X_LON), array(y_LON)
 
# define input sequence
raw_seq_LON = df_fi_d1072339762['LON']*-1
# choose a number of time steps
raw_seq_LON.to_numpy()
raw_seq2_LON=pd.Series(raw_seq_LON).array

print(type(raw_seq2_LON))

n_steps_LON = 4
# split into samples
X_LON, y_LON = split_sequence_LON(raw_seq2_LON, n_steps_LON)
# summarize the data
for i in range(len(X_LON)):
    print(X_LON[i], y_LON[i])

<class 'pandas.core.arrays.numpy_.PandasArray'>
[81.73685 81.73685 81.73684 81.73683] 81.73685
[81.73685 81.73684 81.73683 81.73685] 81.73686
[81.73684 81.73683 81.73685 81.73686] 81.73684
[81.73683 81.73685 81.73686 81.73684] 81.73686
[81.73685 81.73686 81.73684 81.73686] 81.73685
[81.73686 81.73684 81.73686 81.73685] 81.73684
[81.73684 81.73686 81.73685 81.73684] 81.73685
[81.73686 81.73685 81.73684 81.73685] 81.73686
[81.73685 81.73684 81.73685 81.73686] 81.73685
[81.73684 81.73685 81.73686 81.73685] 81.73688
[81.73685 81.73686 81.73685 81.73688] 81.73683
[81.73686 81.73685 81.73688 81.73683] 81.73682
[81.73685 81.73688 81.73683 81.73682] 81.73685
[81.73688 81.73683 81.73682 81.73685] 81.73684
[81.73683 81.73682 81.73685 81.73684] 81.73686
[81.73682 81.73685 81.73684 81.73686] 81.73682
[81.73685 81.73684 81.73686 81.73682] 81.73683
[81.73684 81.73686 81.73682 81.73683] 81.73685
[81.73686 81.73682 81.73683 81.73685] 81.73684
[81.73682 81.73683 81.73685 81.73684] 81.73683
[81.73683 81

In [63]:

n_features = 1
n_seq = 2
n_steps = 2
X_LON = X_LON.reshape((X_LON.shape[0], n_seq, 1, n_steps, n_features))
# define model
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(n_seq, 1, n_steps, n_features)))
model.add(Flatten())
model.add(layers.Dense(1, activation='relu'))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_LON, y_LON, epochs=50, verbose=0)






# x_input_LON = X_LON[(len(raw_seq2_LON)-n_steps_LON-3)]
# x_input_LON = x_input_LON.reshape((1, n_seq, 1, n_steps, n_features))
# yhat_LON = model.predict(x_input_LON, verbose=0)
# print(yhat_LON)

In [64]:
model.fit(X_LON, y_LON, epochs=50, verbose=0)

In [62]:
yhat_LON

array([[81.662834]], dtype=float32)

In [ ]:
geo=[Point(xy) for xy in yhat,yhat_LON]
gdf = GeoDataFrame(df, geometry = geo)
df_fi_d1072339762_111
